# The Perceptron Pocket Algorithm

Many instances of data are linearily seperable, but only after removing noise; the following image is one such example.

![Example of "Noise"](linear_noisy_data.png)

Such data is actually encountered often: even though a linear classifier seems appropiate, the data may not be linearily seperable because of outliers or noise. Notably, we proved that the perceptron learning algorithm (PLA) will not converge whenever the training data is not linearily seperable. Thus, we must exclude the PLA from our toolkit in this instance. More specifically, we need solve the combinatorial optimization problem:

$$
\min_{\boldsymbol{w}\in\mathbb{R}^{d+1}}\frac{1}{N}\sum_{i=n}^{N}||\text{sign}(w^{T}\boldsymbol{x}_n) \neq y_n||.
$$

Note the in-sample error is given by:
$$
E_{\text{in}}(\boldsymbol{w}) = \frac{1}{N}\sum_{i=n}^{N}||\text{sign}(w^{T}\boldsymbol{x}_n) \neq y_n||.
$$

In general, determining the solution the above mentioned optimization problemn is NP-hard. Thus, we will only approximate the solution the minimization problem. Our algorithm is called the pocket algorithm. Essentially, the pocket algorithm keeps in its 'pocket' the best weight vector encountered up to iteration $t$ in PLA. At the end of the algorithm, the best weight vector will be reported as the final hypothesis.

### The Pocket Algorithm

1. Set the pocket weight vector $\hat{\boldsymbol{w}}$ to $\boldsymbol{w}(0)$ of PLA.
1. for $t=0,\dots,T-1$ do
* Run PLA for one update to obtain $\boldsymbol{w}(t+1)$.
* Evaluate $E_{\text{in}}(\boldsymbol{w}(t+1))$.
* if $\boldsymbol{w}(t+1)$ is better than $\hat{\boldsymbol{w}}$ in terms of $E_{\text{in}}(\boldsymbol{w}(t+1))$, set $\hat{\boldsymbol{w}}$ to $\boldsymbol{w}(t+1)$.
1. Return $\hat{\boldsymbol{w}}$

The original PLA only checks some of the examples using $\boldsymbol{w}(t)$ to identify $(\boldsymbol{x}(t), y(t))$ in each iteration, while the pocket algorithm needs an additional step that evaluates all examples using $\boldsymbol{w}(t+1)$ to get $E_{\text{in}}(\boldsymbol{w}(t+1))$. This additional step makes the pocket algorithm much slower that PLA. Nevertheless, it is a useful algorithm to know about. We will next implement the pocket algorithm to classify hand written images of digits in $\{1,5\}$ from the famous *mnist data set.*

## MNIST Data Set
The MNIST data set consists of $70,000$ images of hand written digits, $60,000$ of which are typically used as labeled training examples, where the other $10,000$ are used for testing your learning model on. 

![MNIST Examples](MnistExamples.png)


To access this data set, as well as view the data as an image, we will need the following packages:
* MLDatasets [documentation](https://github.com/JuliaML/MLDatasets.jl)
* Images, Testimages, ImageMagicO [documentation](https://juliaimages.org/latest/)

In [ ]:
using MLDatasets, Images, TestImages

train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata();

The  data structures train_x and test_x are stored as 3 dimensional tensors.

![3-D Tensor](order-3-tensor.png)

In [ ]:
# Show the dimensions of the train_x tensor.
size(train_x)

In [ ]:
# Verify the dimensions of the training vector. This should match the depth of out training 
# tensor. 
size(train_y)

In [ ]:
# Verify the the shape of the elements in the tensor slices.
train_x[:,:,1]

Each image in the is comprised of a $28\times 28$ grey scaled grid of pixel values. These values are floating point numbers in the interval $(0,1)$, where darker pixels will have values closer to $1$ and lighter pixels will have values closer to $0$. The following image represents one such example.

![MNIST Matrix](MNIST-Matrix.png)

In [ ]:
# What are the entries in each of these 28x28 matrices??
train_x[:,:,1]

In [ ]:
# Check the label of the above matrix. 
train_y[1]

In [ ]:
# View the image
colorview(Gray, train_x[:,:,1]')
savefig("MNIST_Example")

In [ ]:
# Use the pocket algorithm to classify images of numbers in the set {1,5}. Note that this is a
# binary classification problem, even though the input vectors have dimension much larger than
# we have seen before. 

# Reduce the dimension of the input data
pocket_train_x = []
pocket_train_y = []
for i = 1:60000
    if train_y[i] == 1 || train_y[i] == 5
        push!(pocket_train_x, reshape(train_x[:,:,i], 784))
        push!(pocket_train_y, train_y[i] == 1 ? 1 : -1)
    end
end

# Handy-dandy sign function
function sign(w, x)
    x = push!(copy(x), 1.0)
    return w'x > 0 ? 1 : -1
end

# Define the in-sample error
function errors(w)
    return sum([sign(w, pocket_train_x[i])!= pocket_train_y[i] ? 1 : 0 
            for i = 1:size(pocket_train_x)[1]])/size(pocket_train_x)[1]
end

In [ ]:
# Define the pocket algorithm on T iterations
function pocket_algorithm(T)
    w = randn(785)
    w_hat = copy(w)
    for i = 1:T
        for j = 1:size(pocket_train_x)[1]
            if sign(w, pocket_train_x[j]) != pocket_train_y[j]
                w += pocket_train_y[j]*push!(copy(pocket_train_x[j]),1.0)
            end
        end
        if errors(w) < errors(w_hat)
                    w_hat = copy(w)
        end
    end
    return w_hat
end

In [ ]:
# Let's try it!
#w = rand(785)
w_new = pocket_algorithm(500);

In [ ]:
indicies = []
for i in 1:size(test_y)[1]
    if test_y[i] == 1 || test_y[i] == 5
        push!(indicies, (i, test_y[i]))
    end
end

In [ ]:
function check_value(i)
    println("Predicted value:", 
        sign(w_new, reshape(test_x[:,:,indicies[i][1]], 784)) == 1 ? 1 : 5, "\n")
    println("Labeled value:", test_y[indicies[i][1]], "\n")
    println("Image:")
    colorview(Gray, test_x[:,:,indicies[i][1]]')
end

In [ ]:
check_value(100)

In [ ]:
check_value(2)

In [ ]:
check_value(3)

In [ ]:
check_value(4)

In [ ]:
check_value(323)

In [ ]:
check_value(129)

In [ ]:
check_value(1700)

In [ ]:
check_value(1200)